# Stock Selection for the Defensive Investor
**Purpose.** Purpose of this notebook is to select companies that pass Ben Graham's test criteria for the *defensive investor*, as described in *Chapter 14 of Intelligent Investor*.
The criteria has 7 tests:
1. **Adequate Size of the Enterprise.** Not less than \\$100 million of annual sales for an industrial company and, not less than \\$50 million of total assets for a public utility.


2. **A Sufficiently Strong Financial Condition.** For industrial companies current assets should be at least twice current liabilities—a so-called two-to-one current ratio. Also, long-term debt should not exceed the net current assets (or “working capital”). For public utilities the debt should not exceed twice the stock equity (at book value).


3. **Earnings Stability.** Some earnings for the common stock in each of the past ten years.


4. **Dividend Record.** Uninterrupted payments for at least the past 20 years.


5. **Earnings Growth.** A minimum increase of at least one-third in per-share earnings in the past ten years using three-year averages at the beginning and end.


6. **Moderate Price/Earnings Ratio.** Current price should not be more than 15 times average earnings of the past three years.


7. **Moderate Ratio of Price to Assets.** Current price should not be more than 1½ times the book value last reported. However, a multiplier of earnings below 15 could justify a correspondingly higher multiplier of assets. As a rule of thumb we suggest that the product of the multiplier times the ratio of price to book value should not exceed 22.5.

In [130]:
import sys
import pandas as pd
import numpy as np
import quandl
import os
import warnings
from tqdm import tqdm_notebook as tqdm
#import humanize

warnings.filterwarnings('ignore')
quandl.ApiConfig.api_key = os.getenv('QUANDL_APY_KEY')
print(f'Python version: {sys.version}')
print(f'Pandas version: {pd.__version__}\nNumpy version: {np.__version__}')

Python version: 3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas version: 0.23.4
Numpy version: 1.15.4


## Getting necessary data for a given company, today
First step is to get necessary data to determine if a given company pass or fails Ben Graham's test criteria for the *defensive investor*, as described in *Chapter 14 of Intelligent Investor*. Using [Quandl SF1 | Core US Fundamentals Data](https://www.quandl.com/databases/SF1)

In [55]:
ticker = 'MSFT'

def get_data(ticker): 
    columns = ['calendardate', 'datekey', 'revenue', 'currentratio', 'eps', 'dps', 'bvps', 'price']
    df = quandl.get_table('SHARADAR/SF1', dimension='MRY', ticker=ticker, qopts={"columns":columns})
    df.sort_values('calendardate', inplace=True)
    return df

get_data(ticker).tail()

,calendardate,datekey,revenue,currentratio,eps,dps,bvps,price
None,,,,,,,,
4,2014-12-31,2014-06-30,86833000000,2.504,2.66,1.07,10.890,41.70
3,2015-12-31,2015-06-30,93580000000,2.473,1.49,1.21,9.932,44.15
2,2016-12-31,2016-06-30,91154000000,2.353,2.59,1.39,9.179,51.17
1,2017-12-31,2017-06-30,96571000000,2.919,3.29,1.53,11.367,68.93
0,2018-12-31,2018-06-30,110360000000,2.901,2.15,1.65,10.768,98.61


## Check if a given company pass the 7 criteria
Second step is to implement the criteria and check if a given company pass it or not.

In [47]:
def criteria_defensive_investor(ticker_or_list, verbose=False):
    list_tickers = []
    if isinstance(ticker_or_list, str):
        list_tickers.append(ticker_or_list)
    else:
        list_tickers = ticker_or_list
        
    summary_columns = ['ticker', 'last_date', 'first_date', 'revenue', 'current_ratio', 
                       'positive_eps_p10yrs_count', 'dividend_distribution_p20yrs_count', 
                       'earnings_change_p10yrs', 'pe', 'pb', 'pexpb', 'size_criteria', 
                       'financial_condition_criteria', 'earnings_stability_criteria', 
                       'dividend_record_criteria', 'earnings_growth_criteria', 'pe_criteria', 
                       'pb_criteria', 'full_criteria']
    df_ = pd.DataFrame(columns=summary_columns)
        
    for ticker in tqdm(list_tickers):
        if verbose:
            print('\nTest for ' + ticker + ':')
        data = get_data(ticker)

        # In case of no data
        if data.shape[0] == 0:  # No data
            if verbose:
                print('- No data available')
            df_.append(pd.Series([float('NaN')] * len(summary_columns), index=summary_columns), ignore_index=True)
            df_.at[df_.index[-1], 'ticker'] = ticker
            continue

        # Size criteria
        size_criteria = data['revenue'].values[-1] > 100000000

        # Financial condition criteria
        if data['currentratio'].values[-1] is not None:
            current_ratio = data['currentratio'].values[-1]
            financial_condition_criteria = current_ratio > 2
        else:
            current_ratio = float('NaN')
            financial_condition_criteria = False

        # Earnings stability criteria
        earnings_stability_criteria = (data['eps'].tail(10) > 0).all()

        # Dividends record criteria
        dividend_record_criteria = (data['dps'].tail(20) > 0).all()

        # Earnings growth criteria
        last_year = pd.to_datetime(data['calendardate'].values[-1]).year
        eps_0 = data[(data['calendardate'].dt.year > last_year - 13) & 
                     (data['calendardate'].dt.year <= last_year - 10)]['eps'].mean()
        eps_1 = data[(data['calendardate'].dt.year > last_year - 3) & 
                     (data['calendardate'].dt.year <= last_year)]['eps'].mean()
        earnings_growth_criteria = (np.float64(eps_1) / eps_0) > 1.33

        # P/E ratio criteria
        current_price = data['price'].values[-1]
        pe = current_price / eps_1
        pe_criteria = pe < 15

        # Price to Assets criteria
        pb = current_price / data['bvps'].values[-1]
        pb_criteria = pb < 1.5
        if (pe * pb < 22.5):
            pb_criteria = True

        # Full criteria
        full_criteria = size_criteria and financial_condition_criteria and earnings_stability_criteria \
            and dividend_record_criteria and earnings_growth_criteria and pe_criteria
        
        # Add to dataframe
        my_dic = {'ticker': ticker, 
                  'last_date': data['datekey'].values[-1], 
                  'first_date': data['datekey'].values[0], 
                  'revenue': data['revenue'].values[-1], 
                  'current_ratio': current_ratio, 
                  'positive_eps_p10yrs_count': data.tail(10)[data['eps'] > 0]['eps'].count(), 
                  'dividend_distribution_p20yrs_count': data.tail(20)[data['dps'] > 0]['dps'].count(), 
                  'earnings_change_p10yrs': (np.float64(eps_1) / eps_0), 
                  'pe': pe, 
                  'pb': pb, 
                  'pexpb': pe * pb,
                  'size_criteria': size_criteria, 
                  'financial_condition_criteria': financial_condition_criteria, 
                  'earnings_stability_criteria': earnings_stability_criteria, 
                  'dividend_record_criteria': dividend_record_criteria, 
                  'earnings_growth_criteria': earnings_growth_criteria, 
                  'pe_criteria': pe_criteria, 
                  'pb_criteria': pb_criteria, 
                  'full_criteria': full_criteria}
        df_.loc[len(df_)] = my_dic 

        if verbose:
            print('- Size criteria: \t\t' + str(size_criteria) + 
                  '\tRevenues of $' + humanize.intword(data['revenue'].values[-1]) + ' (threshold is $100 million)')
            print('- Financial condition criteria: ' + str(financial_condition_criteria) + 
                  '\tCurrent ratio of %1.2f' % current_ratio + ' (threshold is 2.0)')
            print('- Earnings stability criteria: \t' + str(earnings_stability_criteria) + 
                  '\tPositive earnings in %d of past 10 years' % data.tail(10)[data['eps'] > 0]['eps'].count())
            print('- Dividend record criteria: \t' + str(dividend_record_criteria) + 
                  '\tDistribution of dividend in %d of past 20 years' % data.tail(20)[data['dps'] > 0]['dps'].count())
            print('- Earnings growth criteria: \t' + str(earnings_growth_criteria) + 
                  '\tEarnings change of %+.0f%%' % (100 * ((np.float64(eps_1) / eps_0) - 1)) + ' in past 10 years (minimum is +33%)')
            print('- Moderate P/E ratio criteria: \t' + str(pe_criteria) + 
                  '\tCurrent price is %1.1fx avg P3yrs earnings (limit is 15)' % pe)
            print('- Moderate P/B ratio criteria: \t' + str(pb_criteria) + 
                  '\tCurrent price is %1.1fx last book value (limit 1.5), \n\t\t\t\t\tand PE * PB is %1.1f (limit 22.5)' % (pb, pe * pb))
            print('- Full criteria: \t\t' + str(full_criteria))
    
    return df_


In [56]:
criteria_defensive_investor('AAPL').transpose()

,0
ticker,AAPL
last_date,2018-09-29 00:00:00
first_date,1995-09-26 00:00:00
revenue,265595000000
current_ratio,1.124
positive_eps_p10yrs_count,10
dividend_distribution_p20yrs_count,7
earnings_change_p10yrs,15.5538
pe,22.8559
pb,10.1161


## Bulk test it in DJIA companies

In [101]:
def get_djia_symbols():
    data = pd.read_html("https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average")
    table = data[1]
    sliced_table = table[1:]
    header = table.iloc[0]
    corrected_table = sliced_table.rename(columns=header)
    return corrected_table['Symbol'].tolist()

In [102]:
df_djia = criteria_defensive_investor(get_djia_symbols())
df_djia.head()

,ticker,last_date,first_date,revenue,current_ratio,positive_eps_p10yrs_count,dividend_distribution_p20yrs_count,earnings_change_p10yrs,pe,pb,pexpb,size_criteria,financial_condition_criteria,earnings_stability_criteria,dividend_record_criteria,earnings_growth_criteria,pe_criteria,pb_criteria,full_criteria
0,MMM,2017-12-31,1993-12-31,31657000000,1.857,10,20,1.621984,29.178099,12.138105,354.166840,True,False,True,True,True,False,False,False
1,AXP,2017-12-31,1991-12-31,30712000000,NaN,10,20,1.445732,21.715015,4.801992,104.275330,True,False,True,True,True,False,False,False
2,AAPL,2018-09-29,1995-09-26,265595000000,1.124,10,7,15.553806,22.855889,10.116065,231.211672,True,False,True,False,True,False,False,False
3,BA,2017-12-31,1995-12-31,93392000000,1.141,10,20,2.497400,30.698473,106.082734,3256.577970,True,False,True,True,True,False,False,False
4,CAT,2017-12-31,1995-12-31,45462000000,1.346,9,20,0.356246,87.706865,6.774429,594.163953,True,False,False,True,False,False,False,False


### Analysis
Let's analyze how many companies pass stock selection criteria for the defensive investor as proposed by Ben Graham, today.

In [107]:
def analysis(df):
    c1 = len(df[df['size_criteria'] == True]['ticker'].tolist())
    c2 = len(df[df['financial_condition_criteria'] == True]['ticker'].tolist())
    c3 = len(df[df['earnings_stability_criteria'] == True]['ticker'].tolist())
    c4 = len(df[df['dividend_record_criteria'] == True]['ticker'].tolist())
    c5 = len(df[df['earnings_growth_criteria'] == True]['ticker'].tolist())
    c6 = len(df[df['pe_criteria'] == True]['ticker'].tolist())
    c7 = len(df[df['pb_criteria'] == True]['ticker'].tolist())

    c8 = len(df[(df['size_criteria'] == True) & 
                (df['financial_condition_criteria'] == True) & 
                (df['earnings_stability_criteria'] == True) & 
                (df['dividend_record_criteria'] == True) & 
                (df['earnings_growth_criteria'] == True)]['ticker'].tolist())

    cdi = len(df[df['full_criteria'] == True]['ticker'].tolist())

    print(f'{c1:4d} companies passing size criteria of minimum $100 million revenues')
    print(f'{c2:4d} companies passing financial condition criteria of minimum 2 current ratio')
    print(f'{c3:4d} companies passing earnings stability criteria of positive earnings in past 10 years')
    print(f'{c4:4d} companies passing dividend record criteria of uninterrupted payments in past 20 years')
    print(f'{c5:4d} companies passing earnings growth criteria of minimum 33% growth in past 10 years')
    print(f'{c6:4d} companies passing moderate PE ratio criteria of maximum 15')
    print(f'{c7:4d} companies passing moderate PB ratio criteria of maximum 1.5, or PE * PB of maximum 22.5')
    print(f'{c8:4d} companies passing all except moderate PE and PB ratio criteria')
    print(f'{cdi:4d} companies passing all criteria')
    
    df['count_successful_criteria'] = df.apply(lambda row: sum([row['size_criteria'], 
                                                                row['financial_condition_criteria'], 
                                                                row['earnings_stability_criteria'], 
                                                                row['dividend_record_criteria'], 
                                                                row['earnings_growth_criteria'], 
                                                                row['pe_criteria'], 
                                                                row['pb_criteria']]), axis=1)


In [104]:
analysis(df_djia)

df_djia[(df_djia['size_criteria'] == True) & 
        (df_djia['financial_condition_criteria'] == True) & 
        (df_djia['earnings_stability_criteria'] == True) & 
        (df_djia['dividend_record_criteria'] == True) & 
        (df_djia['earnings_growth_criteria'] == True)]['ticker'].tolist()

  30 companies passing size criteria of minimum $100 million revenues
   3 companies passing financial condition criteria of minimum 2 current ratio
  27 companies passing earnings stability criteria of positive earnings in past 10 years
  25 companies passing dividend record criteria of uninterrupted payments in past 20 years
  20 companies passing earnings growth criteria of minimum 33% growth in past 10 years
   3 companies passing moderate PE ratio criteria of maximum 15
   5 companies passing moderate PB ratio criteria of maximum 1.5, or PE * PB of maximum 22.5
   1 companies passing all except moderate PE and PB ratio criteria
   0 companies passing all criteria


['NKE']

As we can see, no company pass all 7 criteria. **Nike** is the closest one, passing all first 5, but failing the last 2 criteria, which are related to company's current price attractiveness.

Now, let's count how many criteria each company pass and analyze that.

In [105]:
df_djia.sort_values('count_successful_criteria', ascending=False)[['ticker', 'count_successful_criteria']]

,ticker,count_successful_criteria
22,TRV,6
15,JPM,5
19,NKE,5
16,MCD,5
12,IBM,5
1,AXP,4
28,WBA,4
27,WMT,4
25,VZ,4
24,UTX,4


## Bulk test it in S&P500 companies

In [108]:
def get_sp500_symbols():
    data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    table = data[1]
    sliced_table = table[1:]
    header = table.iloc[0]
    corrected_table = sliced_table.rename(columns=header)
    return corrected_table['Symbol'].tolist()

In [109]:
df_sp500 = criteria_defensive_investor(get_sp500_symbols())
df_sp500.head()

,ticker,last_date,first_date,revenue,current_ratio,positive_eps_p10yrs_count,dividend_distribution_p20yrs_count,earnings_change_p10yrs,pe,pb,pexpb,size_criteria,financial_condition_criteria,earnings_stability_criteria,dividend_record_criteria,earnings_growth_criteria,pe_criteria,pb_criteria,full_criteria
0,MMM,2017-12-31,1993-12-31,31657000000,1.857,10,20,1.621984,29.178099,12.138105,354.166840,True,False,True,True,True,False,False,False
1,ABT,2017-12-31,1995-12-31,27390000000,2.261,10,20,0.737123,41.255422,3.207982,132.346651,True,True,True,True,False,False,False,False
2,ABBV,2017-12-31,2009-12-31,28216000000,1.275,8,5,NaN,28.697329,30.278647,868.916319,True,False,False,False,False,False,False,False
3,ABMD,2018-03-31,1996-03-31,593749000,5.837,7,0,-1.351744,187.735484,18.707168,3511.999258,True,True,False,False,False,False,False,False
4,ACN,2018-08-31,1999-08-31,41603428000,1.338,10,13,2.870370,27.269355,10.451258,284.999062,True,False,True,False,True,False,False,False


In [111]:
analysis(df_sp500)

df_sp500[(df_sp500['size_criteria'] == True) & 
         (df_sp500['financial_condition_criteria'] == True) & 
         (df_sp500['earnings_stability_criteria'] == True) & 
         (df_sp500['dividend_record_criteria'] == True) & 
         (df_sp500['earnings_growth_criteria'] == True)]['ticker'].tolist()

 495 companies passing size criteria of minimum $100 million revenues
 107 companies passing financial condition criteria of minimum 2 current ratio
 293 companies passing earnings stability criteria of positive earnings in past 10 years
 241 companies passing dividend record criteria of uninterrupted payments in past 20 years
 272 companies passing earnings growth criteria of minimum 33% growth in past 10 years
 102 companies passing moderate PE ratio criteria of maximum 15
 119 companies passing moderate PB ratio criteria of maximum 1.5, or PE * PB of maximum 22.5
  17 companies passing all except moderate PE and PB ratio criteria
   0 companies passing all criteria


['APD',
 'ALB',
 'AOS',
 'BAX',
 'CTAS',
 'COO',
 'EXPD',
 'FAST',
 'GWW',
 'HAS',
 'ITW',
 'IFF',
 'MLM',
 'NKE',
 'SYK',
 'TXN',
 'TIF']

As we can see, still no company pass all 7 criteria. But now, we can see that **17 companies** passed all first 5, but failed the last 2 criteria, which are related to company's current price attractiveness.

Now, let's count how many criteria each company pass and analyze that.

In [114]:
#df_sp500.sort_values('count_successful_criteria', ascending=False)[['ticker', 'count_successful_criteria']]
df_sp500[df_sp500['count_successful_criteria'] == 6]

,ticker,last_date,first_date,revenue,current_ratio,positive_eps_p10yrs_count,dividend_distribution_p20yrs_count,earnings_change_p10yrs,pe,pb,pexpb,size_criteria,financial_condition_criteria,earnings_stability_criteria,dividend_record_criteria,earnings_growth_criteria,pe_criteria,pb_criteria,full_criteria,count_successful_criteria
11,AFL,2017-12-31,1995-12-31,21667000000,NaN,10,20,2.577419,10.986233,1.399286,15.372880,True,False,True,True,True,True,True,False,6
279,LB,2018-02-03,1997-01-31,12632000000,1.621,10,20,2.218045,12.078814,-17.807346,-215.091616,True,False,True,True,True,True,True,False,6
318,TAP,2017-12-31,1995-12-31,11002800000,0.644,10,20,3.175439,13.602762,1.336580,18.181175,True,False,True,True,True,True,True,False,6
332,NWL,2017-12-31,1995-12-31,14742200000,2.269,9,20,2.050000,11.304878,1.065848,12.049282,True,True,False,True,True,True,True,False,6
438,TMK,2017-12-31,1995-12-31,4155573000,NaN,10,20,3.025953,13.039291,1.680531,21.912927,True,False,True,True,True,True,True,False,6
442,TRV,2017-12-31,1995-12-31,28902000000,NaN,10,20,1.854932,14.143900,1.558686,22.045903,True,False,True,True,True,True,True,False,6
457,UNM,2017-12-31,1995-12-31,11286800000,NaN,10,20,2.405680,13.884486,1.286354,17.860360,True,False,True,True,True,True,True,False,6
477,WFC,2017-12-31,1995-12-31,85861000000,NaN,10,20,1.720056,14.737652,1.440135,21.224206,True,False,True,True,True,True,True,False,6


Here we can see 2 interesting things. From all S&P 500 companies, **8 companies above passed all but 1 criteria** that Ben Graham proposed:
- 5 companies have missing current ratios. We already contacted Quandl team to fix that.
- 3 companies have all data. From those:
    - **NWL (Newell Brands Inc)** got pretty close. It failed on earnings stability criteria, getting positive EPS in 9 out of past 10 years.
    - **LB (Newell Brands Inc)** also got pretty close. It failed on financial condition criteria, getting 1.6 current ratio, below minimum threshold of 2.
    - **TAP (Molson Coors Brewing Company)** failed on financial condition criteria as well, but got far from minimum current ratio threshold with 0.6.

## Next steps:
1. Ask for Quandl help to fix companies without current ratio (implement fallback if not fixed)
2. Parametrize `criteria_defensive_investor` by date, adjusting it to return companies passing criteria in a given past date

S&P500 companies without current ratio in Quandl Sharadar Core US Fundamentals Data:

In [122]:
df_sp500[df_sp500['current_ratio'].isnull()]['ticker'].tolist()

['AMG',
 'AFL',
 'ARE',
 'ALL',
 'AXP',
 'AIG',
 'AMP',
 'AIV',
 'AIZ',
 'AVB',
 'BAC',
 'BK',
 'BBT',
 'BLK',
 'BXP',
 'BHF',
 'COF',
 'SCHW',
 'CB',
 'CI',
 'CINF',
 'C',
 'CFG',
 'CMA',
 'DHI',
 'DE',
 'DLR',
 'DFS',
 'DRE',
 'ETFC',
 'EQR',
 'ESS',
 'RE',
 'EXR',
 'FRT',
 'FITB',
 'BEN',
 'GE',
 'GS',
 'HIG',
 'HCP',
 'HST',
 'HBAN',
 'IVZ',
 'JEF',
 'JPM',
 'KEY',
 'KIM',
 'LEN',
 'LIN',
 'L',
 'MTB',
 'MAC',
 'MET',
 'MAA',
 'MS',
 'NTRS',
 'PBCT',
 'PNC',
 'PFG',
 'PGR',
 'PLD',
 'PRU',
 'PSA',
 'PHM',
 'RJF',
 'O',
 'REG',
 'RF',
 'SPG',
 'SLG',
 'STT',
 'STI',
 'SIVB',
 'SYF',
 'TROW',
 'TMK',
 'TRV',
 'UDR',
 'USB',
 'UNM',
 'VTR',
 'VNO',
 'WFC',
 'WELL',
 'WU',
 'ZION']